In [2]:
import numpy as np
import pandas as pd

# Load ratings Change directory where appropriate
directory = 'Data/'
ratings = pd.read_csv(directory+'ratings.csv', usecols=['userId', 'movieId', 'rating'])
movies = pd.read_csv(directory+'movies.csv', usecols=['movieId', 'title', 'genres'])
max_userid = ratings['userId'].drop_duplicates().max()+1
max_movieid = ratings['movieId'].drop_duplicates().max() +1
ratings.head()
print(max_userid)
print(max_movieid)

611
193610


In [3]:

from keras.layers import Embedding, Reshape, Concatenate,dot,Input,Dense
from keras.models import Model,clone_model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

factors = 100 # The number of dimensional embeddings for movies and users


# Create training set
#shuffled_ratings = ratings.sample(frac=1)

# Shuffling users
#user_ids = shuffled_ratings['userId'].values

# Shuffling movies
#movie_ids = shuffled_ratings['movieId'].values

# Shuffling ratings
#ratings_list = shuffled_ratings['rating'].values


user_ids = ratings['userId'].drop_duplicates().values
movie_ids = ratings['movieId'].drop_duplicates().values
ratings_list = ratings['rating'].drop_duplicates().values
user_ids


Using TensorFlow backend.
C:\Users\Steve(Darren)\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Steve(Darren)\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Steve(Darren)\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dt

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [4]:
# user_layers is the embedding layer that creates an User by latent factors matrix.

user_input = Input(shape=(1,), name = 'user_layers')
user_middle  = Embedding(max_userid, factors,input_length=1)(user_input)
user_layers  = Reshape(target_shape=(factors,))(user_middle)


# movie_layers is the embedding layer that creates a Movie by latent factors matrix.
movie_input = Input(shape=(1,), name = 'movie_layers')
movie_middle  = Embedding(max_movieid, factors,input_length=1)(movie_input)
movie_layers  = Reshape(target_shape=(factors,))(movie_middle)

output = dot([user_layers,movie_layers], axes=1)

model = Model(inputs=[user_input,movie_input], 
                  outputs=output)
training_model = clone_model(model)
#model = Model(inputs=[user_layers, movie_layers],output=dot_product)
#model.Dot(left, right)
model.compile(loss='mse', optimizer='adamax')
# Callbacks monitor the validation loss
# Save the model weights each time the validation loss has improved


W0801 22:47:57.601092 14456 deprecation_wrapper.py:119] From C:\Users\Steve(Darren)\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0801 22:47:57.617092 14456 deprecation_wrapper.py:119] From C:\Users\Steve(Darren)\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0801 22:47:57.618092 14456 deprecation_wrapper.py:119] From C:\Users\Steve(Darren)\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0801 22:47:57.680090 14456 deprecation_wrapper.py:119] From C:\Users\Steve(Darren)\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimiz

In [ ]:
callbacks = [EarlyStopping('val_loss', patience=2),  
    ModelCheckpoint('movie_weights.h5', save_best_only=True)]
history = model.fit([user_ids,movie_ids], ratings_list, epochs=30, validation_split=.1, verbose=2, callbacks=callbacks)

In [ ]:
import math
min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
print( 'Minimum RMSE at epoch' + '{:d}'.format(idx+1) + ' = '+ '{:.4f}'.format(math.sqrt(min_val_loss)))

In [5]:
# Use the pre-trained model
trained_model = clone_model(model)
# Load weights
trained_model.load_weights('movie_weights.h5')

W0801 22:48:05.340540 14456 deprecation_wrapper.py:119] From C:\Users\Steve(Darren)\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0801 22:48:05.341540 14456 deprecation_wrapper.py:119] From C:\Users\Steve(Darren)\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [6]:
def predict_rating(user_id, movie_id):
    return trained_model.predict([np.array([user_id]), np.array([movie_id])])[0][0]


In [27]:
from random import choice
rand_user = 450#choice(user_ids)
rand_movie = choice(movie_ids)

user_ratings = ratings[ratings['userId'] == rand_user][['userId', 'movieId', 'rating']]
print(ratings[ratings['userId'] == rand_user][['userId', 'movieId','rating']])
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(rand_user, x['movieId']), axis=1)
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(movies, 
                                                on='movieId', 
                                                how='inner', 
                                                suffixes=['_u', '_m']).head(20)

#print(user_ratings['prediction'].values.shape)


       userId  movieId
70557     450       70
70558     450      110
70559     450      111
70560     450      260
70561     450      358
70562     450      524
70563     450      551
70564     450      589
70565     450      648
70566     450      908
70567     450     1035
70568     450     1073
70569     450     1193
70570     450     1206
70571     450     1259
70572     450     1387
70573     450     1466
70574     450     1544
70575     450     1580
70576     450     1673
70577     450     1969
70578     450     1972
70579     450     1997
70580     450     2028
70581     450     2291
70582     450     2858
70583     450     2924
70584     450     3173
70585     450     3354
70586     450     3363
70587     450     3404
70588     450     3409
70589     450     3471
70590     450     3481
70591     450     3510
70592     450     3535
70593     450     3568
70594     450     3624
70595     450     3671
70596     450     3753
70597     450     3863
70598     450     3893
70599     4

,userId,movieId,rating,prediction,title,genres
0,450,1673,5.0,4.667173,Boogie Nights (1997),Drama
1,450,1193,5.0,4.759022,One Flew Over the Cuckoo's Nest (1975),Drama
2,450,3972,5.0,4.933046,"Legend of Drunken Master, The (Jui kuen II) (1...",Action|Comedy
3,450,260,5.0,4.756913,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
4,450,1387,5.0,4.779937,Jaws (1975),Action|Horror
5,450,2924,5.0,4.849408,Drunken Master (Jui kuen) (1978),Action|Comedy
6,450,110,5.0,4.904960,Braveheart (1995),Action|Drama|War
7,450,589,5.0,4.576507,Terminator 2: Judgment Day (1991),Action|Sci-Fi
8,450,2028,5.0,4.817351,Saving Private Ryan (1998),Action|Drama|War
9,450,3510,5.0,4.152579,Frequency (2000),Drama|Thriller


In [ ]:
recommendations = ratings[ratings['movieId'].isin(user_ratings['movieId']) == False][['movieId']].drop_duplicates()
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(rand_user, x['movieId']), axis=1)
recommendations.sort_values(by='prediction',
                          ascending=False).merge(movies,
                                                 on='movieId',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(20)

In [35]:
matrix = np.zeros((movie_ids.size,1))
#print(matrix)
for user in user_ids:
    newcol = np.array([predict_rating(user, movie) for movie in movie_ids])
    newcol.shape=((movie_ids.size,1))
    matrix = np.hstack((matrix,newcol))
matrix = matrix[:,1:]
print(matrix)


[[ 4.23139238e+00  3.89563203e+00  1.78582907e+00 ...  1.37252789e-02
   1.98703930e-02 -1.14901895e-02]
 [ 4.05474377e+00  3.18939471e+00  1.94950473e+00 ... -1.55227697e-02
   3.70850302e-02  2.40895655e-02]
 [ 4.64095354e+00  3.97721124e+00  2.05119467e+00 ...  5.32610714e-03
   3.57479863e-02 -1.31487902e-02]
 ...
 [-5.93150668e-02 -1.23669349e-01 -5.30145653e-02 ...  3.08892131e-03
  -1.10348687e-02 -1.35623524e-02]
 [ 2.18230113e-03 -4.88747731e-02  3.20992768e-02 ...  4.36150748e-03
  -4.75833984e-03 -1.32972980e-03]
 [-5.59398457e-02 -2.51321513e-02  7.71414936e-02 ... -1.37456097e-02
   2.09657345e-02  1.01437327e-05]]
